# BERT on Repo Description

1. Construct a sentence corpus for each software type using labeled/manually validated repo descriptions
2. Calculate embeddingfor each corpus
3. Compare all repo description with each software type corpus using cosine-similarity score
    - It took about 1.5 hrs to run the embedding on repo data

Author: Cierra and Crystal

In [67]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [95]:
#pgadmin
import os
import psycopg2 as pg


#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd

import re

import nltk
nltk.download("punkt")

from nltk import tokenize

import scipy

import datetime

[nltk_data] Downloading package punkt to /home/zz3hs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Functions

In [69]:
#TODO: work on the cleaning function
def clean_text(texts):
    texts = re.sub(r'\[[0-9]*\]', ' ', texts) #removes numbers
   # texts = re.sub(r'\s+',' ',texts) #remove long blank space
    texts = texts.lower() #lowercase text
   # texts = re.sub(r'\d', ' ', texts) 
   # texts = re.sub(r'\s+', ' ', texts)
    return(texts)

# Embedding Model

In [70]:
#embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2') #quicker model
embedder = SentenceTransformer('paraphrase-mpnet-base-v2') #most accurate, long run time

# Data

## I. Unlabelled Repo Data

In [72]:
os.environ['db_user'] = 'zz3hs'
os.environ['db_pwd'] = 'zz3hs'

repo_data=[]

# connect to the database, download data, limit to repos with at least 20,000 commits?
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))

repo_data = '''SELECT * FROM gh_2007_2020.repos_subset_358k'''
#repo_data = '''SELECT * FROM gh_2007_2020.repos_subset_157k'''

# convert to a dataframe, show how many missing we have (none)
repo_data = pd.read_sql_query(repo_data, con=connection)

connection.close()

In [73]:
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers
0,vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adopt...","<p align=""center""><a href=""https://vuejs.org"" ...",JavaScript,"['vue', 'javascript', 'frontend', 'framework']",3070.0,29611.0,185611.0,6250.0
1,facebook/react,"A declarative, efficient, and flexible JavaScr...",# [React](https://reactjs.org/) · [![GitHub li...,JavaScript,"['javascript', 'react', 'frontend', 'declarati...",12695.0,34352.0,171327.0,6718.0
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,"<div align=""center"">\n<img src=""https://www.te...",C++,"['tensorflow', 'machine-learning', 'python', '...",75671.0,84937.0,156754.0,8092.0
3,twbs/bootstrap,"The most popular HTML, CSS, and JavaScript fra...","<p align=""center"">\n<a href=""https://getbootst...",JavaScript,"['css', 'bootstrap', 'javascript', 'html', 'sc...",19228.0,73981.0,151778.0,7079.0
4,ohmyzsh/ohmyzsh,🙃 A delightful community-driven (with 1700+ c...,"<p align=""center""><img alt=""Oh My Zsh"" src=""ht...",Shell,"['shell', 'zsh-configuration', 'theme', 'termi...",5447.0,22232.0,129314.0,2678.0
...,...,...,...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototy...,"# CA Alerts, made with Very Little Gravitas fo...",CSS,"['prototype', 'messaging', 'emergency', 'govte...",414.0,1.0,0.0,2.0
157534,dajinchu/kde-connect-android,For Google Code-In,None,Java,[],414.0,0.0,0.0,1.0
157535,LibrinnoTeam/LibraryHelpBot,Library Management System. ITP2 project,# Library Help Bot\r\n\r\n## Purpose of the ap...,Python,"['telegram-bot', 'mariadb', 'python', 'flask']",415.0,2.0,0.0,2.0
157536,Twissi/Animator,Animator for hacklace. See hacklace.org for fu...,Animator\n========\n\nAnimator for hacklace. S...,Java,[],415.0,0.0,0.0,0.0


In [85]:
repo_description = repo_data["description"].tolist()
repo_description[1:10]

#text cleaning
repo_description_clean = []
for text in repo_description:
    cleaned_text_i= clean_text(text)
    repo_description_clean.append(cleaned_text_i)

print(repo_description_clean[0:10])
len(repo_description_clean)

['🖖 vue.js is a progressive, incrementally-adoptable javascript framework for building ui on the web.', 'a declarative, efficient, and flexible javascript library for building user interfaces.', 'an open source machine learning framework for everyone', 'the most popular html, css, and javascript framework for developing responsive, mobile first projects on the web.', '🙃  a delightful community-driven (with 1700+ contributors) framework for managing your zsh configuration. includes nearly 300 optional plugins (rails, git, osx, hub, capistrano, brew, ant, php, python, etc), over 140 themes to spice up your morning, and an auto-update tool so that makes it easy to keep up with the latest updates from the community.', 'flutter makes it easy and fast to build beautiful apps for mobile and beyond.', '📝 algorithms and data structures implemented in javascript with explanations and links to further readings', 'javascript style guide', 'all algorithms implemented in python', '「java学习+面试指南」一份涵盖大

157538

## II. Labelled Repo Data -- Software Type Corpus

In [77]:
data = pd.read_excel('/home/zz3hs/git/dspg21oss/data/dspg21oss/oss_software_labelled_python_sz.xlsx') #import csv
data

<ipython-input-77-99e2bcdb9902>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  data = pd.read_excel('/home/zz3hs/git/dspg21oss/data/dspg21oss/oss_software_labelled_python_sz.xlsx') #import csv


,slug,description,language,topics,commits,forks,stars,watchers,python_label,prog_python
0,openwrt/openwrt,this repository is a mirror of https://git.ope...,C,NaN,46330,5847,8944,547,NaN,1
1,grpc/grpc,"the c based grpc (c++, python, ruby, objective...",C++,NaN,42676,8069,30940,1359,1.0,1
2,libvirt/libvirt,read-only mirror. please submit merge requests...,C,NaN,35951,496,818,84,NaN,1
3,coreboot/coreboot,mirror of https://review.coreboot.org/coreboot...,C,NaN,32000,367,1309,116,NaN,1
4,ccxt/ccxt,a javascript / python / php cryptocurrency tra...,JavaScript,altcoin api arbitrage bitcoin bot cryptocurren...,27607,4971,20017,951,1.0,1
...,...,...,...,...,...,...,...,...,...,...
594,alfredfrancis/ai-chatbot-framework,a python chatbot framework with natural langua...,Python,chatbot python nltk ai sklearn natural-languag...,355,631,1491,130,1.0,1
595,jhao104/proxy_pool,pythonÁà¨Ëô´‰ª£ÁêÜipÊ±†(proxy pool),Python,crawler proxy proxypool spider ssdb flask craw...,354,3567,12785,438,NaN,1
596,getsentry/responses,a utility for mocking out the python requests ...,Python,NaN,354,269,3259,73,1.0,2
597,spotify/dh-virtualenv,python virtualenvs in debian packages,Python,dh-virtualenv python debian debian-packages om...,353,168,1471,41,NaN,1


In [82]:
type_name =  "python_label"
corpus_type_i = data[data[type_name] ==1][["slug","description", type_name]]

corpus_type_i = corpus_type_i["description"].tolist()

#text cleaning
corpus_type_i_clean = []
for text in corpus_type_i:
    cleaned_text_i= clean_text(text)
    corpus_type_i_clean.append(cleaned_text_i)

corpus_type_i_clean[0:10]

['the c based grpc (c++, python, ruby, objective-c, php, c#)',
 'a javascript / python / php cryptocurrency trading api with support for more than 120 bitcoin/altcoin exchanges',
 'low code open source framework in python and js',
 'flexible and powerful data analysis / manipulation library for python, providing labeled data structures similar to r data.frame objects, statistical functions, and much more',
 'python-based continuous integration testing framework; your pull requests are more than welcome!',
 'interactive data visualization in the browser, from  python',
 'official repository for spyder - the scientific python development environment',
 'numpy aware dynamic python compiler using llvm',
 'a numpy-compatible array library accelerated by cuda',
 'the database toolkit for python']

# Embedding 

In [83]:

corpus_type_i_embeddings = embedder.encode(corpus_type_i_clean, show_progress_bar=True) # embeddings


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [96]:
queries = repo_description_clean

# pre-specified number of sentences
k = 10


# init a result list for scores
result = []

# Find the closest k sentences of the AI corpus for each query sentence (ML) based on cosine similarity
top_k = min(k, len(corpus_type_i_embeddings))


t1 = datetime.datetime.now()
print("Start:", t1)

for query in queries: #compare each sentence in the abstract to the software type corpus
    #Compute embeddings
    query_embedding = embedder.encode(query, show_progress_bar=False, convert_to_tensor=True) 

    # We use cosine-similarity and torch.topk to find the highest k scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_type_i_embeddings)[0]

    top_results = torch.topk(cos_scores, k=top_k)   #get the top k scores
    result.append(top_results.values.tolist()) #unlist the top result list
   
    #print("\n\n======================\n\n")
    #print("Query:", query)
    #print("Results:", top_results)
    #print("\nTop k=10 most similar sentences in corpus:")
   # for score, idx in zip(top_results[0], top_results[1]):
   #     print(corpus_type_i_clean[idx], "(Score: {:.4f})".format(score))

t2 =  datetime.datetime.now()
print("Finished", len(result), "descriptions at", t2)
print("It took", t2-t1, "to run.")

Start: 2021-07-17 15:15:06.160619
Finished 157538 descriptions at 2021-07-17 16:43:47.442722
It took 1:28:41.282103 to run.


In [97]:
repo_data["similarity_score"] = result

In [98]:
repo_data.to_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_data_python_score.csv', index = False)   


In [99]:
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers,similarity_score
0,vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adopt...","<p align=""center""><a href=""https://vuejs.org"" ...",JavaScript,"['vue', 'javascript', 'frontend', 'framework']",3070.0,29611.0,185611.0,6250.0,"[0.42707720398902893, 0.41736093163490295, 0.3..."
1,facebook/react,"A declarative, efficient, and flexible JavaScr...",# [React](https://reactjs.org/) · [![GitHub li...,JavaScript,"['javascript', 'react', 'frontend', 'declarati...",12695.0,34352.0,171327.0,6718.0,"[0.5678911805152893, 0.5504274368286133, 0.549..."
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,"<div align=""center"">\n<img src=""https://www.te...",C++,"['tensorflow', 'machine-learning', 'python', '...",75671.0,84937.0,156754.0,8092.0,"[0.6914295554161072, 0.6374238133430481, 0.602..."
3,twbs/bootstrap,"The most popular HTML, CSS, and JavaScript fra...","<p align=""center"">\n<a href=""https://getbootst...",JavaScript,"['css', 'bootstrap', 'javascript', 'html', 'sc...",19228.0,73981.0,151778.0,7079.0,"[0.4793936014175415, 0.46719563007354736, 0.46..."
4,ohmyzsh/ohmyzsh,🙃 A delightful community-driven (with 1700+ c...,"<p align=""center""><img alt=""Oh My Zsh"" src=""ht...",Shell,"['shell', 'zsh-configuration', 'theme', 'termi...",5447.0,22232.0,129314.0,2678.0,"[0.4871608316898346, 0.44735753536224365, 0.44..."
...,...,...,...,...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototy...,"# CA Alerts, made with Very Little Gravitas fo...",CSS,"['prototype', 'messaging', 'emergency', 'govte...",414.0,1.0,0.0,2.0,"[0.34419283270835876, 0.3314574360847473, 0.32..."
157534,dajinchu/kde-connect-android,For Google Code-In,None,Java,[],414.0,0.0,0.0,1.0,"[0.525587260723114, 0.47061866521835327, 0.459..."
157535,LibrinnoTeam/LibraryHelpBot,Library Management System. ITP2 project,# Library Help Bot\r\n\r\n## Purpose of the ap...,Python,"['telegram-bot', 'mariadb', 'python', 'flask']",415.0,2.0,0.0,2.0,"[0.5185543894767761, 0.517078697681427, 0.4827..."
157536,Twissi/Animator,Animator for hacklace. See hacklace.org for fu...,Animator\n========\n\nAnimator for hacklace. S...,Java,[],415.0,0.0,0.0,0.0,"[0.41843315958976746, 0.4162086844444275, 0.40..."


In [ ]:
# k: number of similar sentences from AI corpus
# abstract: abstract from FEDERAL RePORTER
# print_result: if TRUE, print out the similar sentenses from AI corpus to each sentence in the abstract
def get_score(k, abstract, print_result = False):
    queries = tokenize.sent_tokenize(abstract) 

    # init a result list for scores
    result = []
    
    # Find the closest k sentences of the AI corpus for each query sentence (ML) based on cosine similarity
    top_k = min(k, len(ai_embeddings))
    
    for query in queries: #compare each sentence in the abstract to the ai corpus
        query_embedding = embedder.encode(query, show_progress_bar=False) 
        
        # We use cosine-similarity and torch.topk to find the highest k scores
        cos_scores = util.pytorch_cos_sim(query_embedding, ai_embeddings)[0]
        
        top_results = torch.topk(cos_scores, k=top_k)   #get the top k scores
        result.append(top_results.values.tolist()) #unlist the top result list
        if print_result:
            print("\n\n======================\n\n")
            print("Query:", query)
            print("Results:", top_results)
            print("\nTop k=5 most similar sentences in corpus:")
            for score, idx in zip(top_results[0], top_results[1]):
                print(ai_corpus[idx], "(Score: {:.4f})".format(score))
    return result

In [ ]:
import datetime
start_time = datetime.datetime.now()
print("Start time for calculating embeddings:", start_time)

abstracts = abstracts.assign(score= abstracts["ABSTRACT"].apply(lambda x: get_score(10,x,False)))

end_time = datetime.datetime.now()
print("Finished calculating ", len(abstracts), "of", "embedding score at", end_time)
print("It took", end_time-start_time, "to run.")
